# Assignment 1

In this assignment you will build a language model for the [OHHLA corpus](http://ohhla.com/) we are using in the book. You will train the model on the available training set, and can tune it on the development set. After submission we will run your notebook on a different test set. Your mark will depend on 

* whether your language model is **properly normalized**,
* its **perplexity** on the unseen test set,
* your **description** of your approach. 

To develop your model you have access to:

* The training and development data in `data/ohhla`.
* The code of the lecture, stored in a python module [here](/edit/statnlpbook/lm.py).
* Libraries on the [docker image](https://github.com/uclmr/stat-nlp-book/blob/python/Dockerfile) which contains everything in [this image](https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook), including scikit-learn and tensorflow. 

As we have to run the notebooks of all students, and because writing efficient code is important, **your notebook should run in 5 minutes at most**, on your machine. Further comments:

* We have tested a possible solution on the Azure VMs and it ran in seconds, so it is possible to train a reasonable LM on the data in reasonable time. 

* Try to run your parameter optimisation offline, such that in your answer notebook the best parameters are already set and don't need to be searched.

## Setup Instructions
It is important that this file is placed in the **correct directory**. It will not run otherwise. The correct directory is

    DIRECTORY_OF_YOUR_BOOK/assignments/2017/assignment1/problem/
    
where `DIRECTORY_OF_YOUR_BOOK` is a placeholder for the directory you downloaded the book to. After you placed it there, **rename the file** to your UCL ID (of the form `ucxxxxx`). 

## General Instructions
This notebook will be used by you to provide your solution, and by us to both assess your solution and enter your marks. It contains three types of sections:

1. **Setup** Sections: these sections set up code and resources for assessment. **Do not edit these**. 
2. **Assessment** Sections: these sections are used for both evaluating the output of your code, and for markers to enter their marks. **Do not edit these**. 
3. **Task** Sections: these sections require your solutions. They may contain stub code, and you are expected to edit this code. For free text answers simply edit the markdown field.  

Note that you are free to **create additional notebook cells** within a task section. 

Please **do not share** this assignment publicly, by uploading it online, emailing it to friends etc. 


## Submission Instructions

To submit your solution:

* Make sure that your solution is fully contained in this notebook. 
* **Rename this notebook to your UCL ID** (of the form "ucxxxxx"), if you have not already done so.
* Download the notebook in Jupyter via *File -> Download as -> Notebook (.ipynb)*.
* Upload the notebook to the Moodle submission site.


## <font color='green'>Setup 1</font>: Load Libraries
This cell loads libraries important for evaluation and assessment of your model. **Do not change it.**

In [1]:
#! SETUP 1
import sys, os
_snlp_book_dir = "../../../../"
sys.path.append(_snlp_book_dir) 
import statnlpbook.lm as lm
import statnlpbook.ohhla as ohhla
import math

## <font color='green'>Setup 2</font>: Load Training Data

This cell loads the training data. We use this data for assessment to define the reference vocabulary: the union of the words of the training and set set. You can use the dataset to train your model, but you are also free to load the data in a different way, or focus on subsets etc. However, when you do this, still **do not edit this setup section**. Instead refer to the variables in your own code, and slice and dice them as you see fit.   

In [2]:
#! SETUP 2
_snlp_train_dir = _snlp_book_dir + "/data/ohhla/train"
_snlp_dev_dir = _snlp_book_dir + "/data/ohhla/dev"
_snlp_train_song_words = ohhla.words(ohhla.load_all_songs(_snlp_train_dir))
_snlp_dev_song_words = ohhla.words(ohhla.load_all_songs(_snlp_dev_dir))
assert(len(_snlp_train_song_words)==1041496)

Could not load ../../../..//data/ohhla/train/www.ohhla.com/anonymous/nas/distant/tribal.nas.txt.html


Due to file encoding issues this code produces one error `Could not load ...`. **Ignore this error**.

## <font color='blue'>Task 1</font>: Develop and Train the Model

This is the core part of the assignment. You are to code up, train and tune a language model. Your language model needs to be subclass of the `lm.LanguageModel` class. You can use some of the existing language models developed in the lecture, or develop your own extensions. 

Concretely, you need to return a better language model in the `create_lm` function. This function receives a target vocabulary `vocab`, and it needs to return a language model defined over this vocabulary. 

The target vocab will be the union of the training and test set (hidden to you at development time). This vocab will contain words not in the training set. One way to address this issue is to use the `lm.OOVAwareLM` class discussed in the lecture notes.

In [3]:
# inject OOVs into the training dataset 
oov_train = lm.inject_OOVs(_snlp_train_song_words)
oov_dev = lm.inject_OOVs(_snlp_dev_song_words)
#len(oov_train)---1041496
#len(oov_vocab)   --- 20460

#The sets module provides classes for constructing and manipulating unordered collections of unique elements.
#so here oov_vocab provides a set of unique elements in oov_train, which can be treated as a vocabulary list
oov_vocab = set(oov_train)

###  <font color='orange'>Smoothing algorithms</font>      

In [4]:
from statnlpbook.lm import *
## StupidBackoffNormalized function can fix the normalization problem of the StupidBackoff function            
class StupidBackoffNormalized(LanguageModel):
    def __init__(self, main, backoff, alpha):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff = backoff
        self.alpha = alpha               

    def probability(self, word, *history):
        main_counts = self.main.counts((word,)+tuple(history))
        main_norm = self.main.norm(history)        
        backoff_order_diff = self.main.order - self.backoff.order
        backoff_counts = self.backoff.counts((word,)+tuple(history[:-backoff_order_diff]))
        backoff_norm = self.backoff.norm(history[:-backoff_order_diff])        
        counts = main_counts + self.alpha * backoff_counts
        norm = main_norm + self.alpha * backoff_norm
        return counts / norm


####  <font color='orange'>Interpolation Smoothing</font>     
Based on the given'InterpolatedLM' function, I combining and weighing the trigram, bigram, and unigram counts. This can be represented by following function:


$$
P(w_n|w_{n-2}w_{n-1})=\lambda_1(w_{n-2}^{n-1})P(w_n|w_{n-2}w_{n-1})+\lambda_2(w_{n-2}^{n-1})P(w_n|w_{n-1})+\lambda_3(w_{n-2}^{n-1})P(w)\\
$$


In [5]:
#interplotedLM2 function takes two backoffs, which given similar effect as using InterplotedLM twice
class InterpolatedLM2(LanguageModel):
    def __init__(self, main, backoff1,backoff2, alpha1, alpha2):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff1 = backoff1
        self.backoff2 = backoff2
        self.alpha1 = alpha1
        self.alpha2 = alpha2

    def probability(self, word, *history):
        return self.alpha1 * self.main.probability(word, *history) + \
                self.alpha2 * self.backoff1.probability(word, *history) + \
               (1.0 - self.alpha1 - self.alpha2) * self.backoff2.probability(word, *history)


####  <font color='orange'>Kneser-Ney Smoothing</font>  
For given size of the training dataset, Kneser-Ney Smoothing is supposed to give good performance. Consider the simplification of Kneser-Net Smoothing -- Absolute Discounting. It is similar to interpolation, however, Absolute Discouting Algorithm subtracts a fixed discount $$\delta \in [0,1]$$ 

$$
P_{\mbox{Absolute}}(w|h_{m}) = 
\begin{cases}
\frac {\#_D(h_{m},w)}{\#_D(h_{m})} -d  &= \mbox{if }\#_D{h_{m},w} > 0 \\\\
\alpha(h_{m-1})\cdot P_{\mbox{Absolute}}(w|h_{m-1}) & \mbox{otherwise}
\end{cases}
$$

$\alpha(h_{m-1})$ is a normalizer

$$\alpha(h_{m-1})=\frac{d}{\sum_{w^{'}} c(w_{i-1},w^{'})} | {w^{'}:0 < c(w_{i-1},w^{'})} |
$$

In [6]:
class AbsoultDiscounting(LanguageModel):
    def __init__(self, main, backoff, d):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff = backoff
        self.d = d

    def probability(self, word, *history):
        main_counts = self.main.counts((word,)+tuple(history))
        main_norm = self.main.norm(history)        
        backoff_counts = self.backoff.counts((word,)+tuple(history[1:]))
        backoff_norm = self.backoff.norm(history[:-backoff_order_diff])
        norm = main_norm + self.alpha * backoff_norm
        #lambda = d/ (w_{i-1}: w_{i-1} w_i) 
        if norm == 0:
            return self.backoff.probability(word, *history)
        else:
            return (main_counts - self.d)/backoff_counts + alpha * self.backoff.probability(word, *history)


####  <font color='orange'>Katz Smoothing</font>  
Katz Smoothing applies Good Turning estimates to the problem of backoff language models. It uses a form of discounting, the total number of counts discounted in the global distribution is equal to the total number of counts that should be assigned to N-Grams with zero counts according to Good Turning estimate.

$$
P_{\mbox{Katz}}(w_i|w_{i-1})=
\begin{cases}
\frac{C(w_{i-1}w_i)}{C(W_{i-1})}  &\mbox{r>k}\\\\
d_r\frac{C(w_{i-1}w_i)}{C(W_{i-1})}&\mbox{ k>=r>0}\\\\
\alpha (w_{i-1})P(w_i)&\mbox{r=0}
\end{cases}
$$

where 
$$
r^*=(r+1)\frac{n_{r+1}}{n_r}
$$

$$
d_r=\frac{\frac{r^*}{r}-\frac{(k+1)n_{k+1}}{n_1}}{1-\frac{(k+1)n_{k+1}}{n_1}}
$$

$$
\alpha(w_{i-1})=\frac{1-\sum_{w_i:r>0}P_{Katz}(w_i|w_{i-1})}{1-\sum_{w_i:r>0}P_{Katz}(w_i)}
$$

In [7]:
class KatzSmoothing(LanguageModel):
    def __init__(self, main, backoff,k):
        super().__init__(main.vocab, main.order)
        self.main = main
        self.backoff = backoff
        self.k = k

    def probability(self, word, *history):
        r = counts[word]
        if (r > 0 and r <= self.k):
            
            r_star = (r+1) * (sorted_counts[r+1]/sorted_counts[r])
            dr = ((r_star/r) - ((self.k + 1) * sorted_counts[self.k + 1] / sorted_counts[1])) / \
                (1 - ((self.k+1) * sorted_counts[self.k + 1] / (sorted_counts[1])))
            return dr * self.backoff.counts((word,)+tuple(history[:-1])) / r 
        elif (r > self.k):
            return self.backoff.counts((word,)+tuple(history[:-1])) / r
        else:
            return (1 - np.sum(self.main.probability(word, *history))) / (1 - np.sum(self.backoff.probability(word, *history)))
       

In [8]:
## You should improve this cell
def create_lm(vocab):
    """
    Return an instance of `lm.LanguageModel` defined over the given vocabulary.
    Args:
        vocab: the vocabulary the LM should be defined over. It is the union of the training and test words.
    Returns:
        a language model, instance of `lm.LanguageModel`.
    """

    unigram = lm.LaplaceLM(lm.NGramLM(oov_train, 1),0.4)
    bigram = lm.NGramLM(oov_train, 2)
    trigram = lm.NGramLM(oov_train, 3)
    quadgram = lm.NGramLM(oov_train, 4)
    pentagram = lm.NGramLM(oov_train, 5)
    sixgram = lm.NGramLM(oov_train, 6)
    my_LM1=lm.InterpolatedLM(bigram,unigram,0.7)        
    my_LM2=lm.InterpolatedLM(trigram,my_LM1,0.17)
    my_LM3=lm.InterpolatedLM(quadgram,my_LM2,0.02)
    my_LM4=lm.InterpolatedLM(pentagram,my_LM3,0.04)
    my_LM=lm.InterpolatedLM(sixgram,my_LM4,0.05)
  
    # the unseen words can be achieved by subtract vocab with oov_vocab
    return lm.OOVAwareLM(my_LM, vocab - oov_vocab)
   
    

## <font color='green'>Setup 3</font>: Specify Test Data
This cell defines the directory to load the test songs from. Currently, this points to the dev set but when we evaluate your notebook we will point this directory elsewhere and use a **hidden test set**.  

In [9]:
#! SETUP 3
_snlp_test_dir = _snlp_book_dir + "/data/ohhla/test"

## <font color='green'>Setup 4</font>: Load Test Data and Prepare Language Model
In this section we load the test data, prepare the reference vocabulary and then create your language model based on this vocabulary.

In [10]:
#! SETUP 4
_snlp_test_song_words = ohhla.words(ohhla.load_all_songs(_snlp_test_dir))
_snlp_test_vocab = set(_snlp_test_song_words)
_snlp_dev_vocab = set(_snlp_dev_song_words)
_snlp_train_vocab = set(_snlp_train_song_words)
_snlp_vocab = _snlp_test_vocab | _snlp_train_vocab | _snlp_dev_vocab
_snlp_lm = create_lm(_snlp_vocab)

FileNotFoundError: [Errno 2] No such file or directory: '../../../..//data/ohhla/test'

## <font color='red'>Assessment 1</font>: Test Normalization (20 pts)
Here we test whether the conditional distributions of your language model are properly normalized. If probabilities sum up to $1$ you get full points, you get half of the points if probabilities sum up to be smaller than 1, and 0 points otherwise. Due to floating point issues we will test with respect to a tolerance $\epsilon$ (`_eps`).

Points:
* 10 pts: $\leq 1 + \epsilon$
* 20 pts: $\approx 1$

In [ ]:
#! ASSESSMENT 1
_snlp_test_token_indices = [100, 1000, 10000]
_eps = 0.000001
approx_1 = []
leq_1 = []
for i in _snlp_test_token_indices:
    result = sum([_snlp_lm.probability(word, *_snlp_test_song_words[i-_snlp_lm.order+1:i]) for word in _snlp_vocab])
    approx_1.append(abs(result - 1.0) < _eps)
    leq_1.append(result - _eps <= 1.0)
    
    print("Sum: {sum}, ~1: {approx_1}, <=1: {leq_1}".format(sum=result, 
                                                            approx_1=abs(result - 1.0) < _eps, 
                                                            leq_1=result - _eps <= 1.0))
(sum(approx_1) == 3, sum(leq_1) == 3)


The above solution is marked with **
<!-- ASSESSMENT 2: START_POINTS -->
20
<!-- ASSESSMENT 2: END_POINTS --> 
points **.

### <font color='red'>Assessment 2</font>: Apply to Test Data (50 pts)

We assess how well your LM performs on some unseen test set. Perplexities are mapped to points as follows.

* 0-10 pts: uniform perplexity > perplexity > 550, linear
* 10-30 pts: 550 > perplexity > 140, linear
* 30-50 pts: 140 > perplexity > 105, linear

The **linear** mapping maps any perplexity value between the lower and upper bound linearly to a score. For example, if uniform perplexity is $U$ and your model's perplexity is $P\leq550$, then your score is $10\frac{P-U}{550-U}$. 

In [ ]:
lm.perplexity(_snlp_lm, _snlp_test_song_words)

The above solution is marked with **
<!-- ASSESSMENT 3: START_POINTS -->
29
<!-- ASSESSMENT 3: END_POINTS --> points**. 

## <font color='blue'>Task 2</font>: Describe your Approach

< Enter a 500 words max description of your model and the way you trained and tuned it here >

## <font color='red'>Assessment 3</font>: Assess Description (30 pts) 

We will mark the description along the following dimensions: 

* Clarity (10pts: very clear, 0pts: we can't figure out what you did)
* Creativity (10pts: we could not have come up with this, 0pts: Use the unigram model from the lecture notes)
* Substance (10pts: implemented complex state-of-the-art LM, 0pts: Use the unigram model from the lecture notes)

* Clarity: 10
* Creativity: 4 (2 for interpolated n-grams, 2 for extra methods thought of (katz, kn)
* Substance: 4 (same as creativity)

The above solution is marked with **
<!-- ASSESSMENT 1: START_POINTS -->
18
<!-- ASSESSMENT 1: END_POINTS --> points**. 

### <font color='orange'>Pretreatment of training data</font>
I first use the 'inject_OOV' function, which use a heuristic to inject OOV symbols into a dataset. Here, the first appearance of a word in 'oov_train' will be treated as 'unknow words', being labeled as OOV.

Given new sequence of training data with OOV injected. 'oov_vocab' provides a set of unique elements in 'oov_train', this can be treated as a vocabulary list.

'OOVAvareLM'is used to return a language model in 'create_lm'. Given input argument 'vacab', 'vocab - oov_vocab' represents the words that are included in 'vocab' while not in 'oov_vocab'.

### <font color='orange'>N-Grams and Add-k Smoothing</font> 
For the given training data('oov_train'), the perplexity of either unigram,bigram or trigram are 'inf'. Therefore, I first applied 'lm.LaplaceLM' function (Add-k smoothing) to improve the language model.

By using the 'for loop', the optimal fractional count alpha was found. (with alpha = 0.4,perplexity_unigram = 547.5497189073692; With alpha = 0.004,  perplexity_bigram =300.97971886348046). Which is worth mentioning is that,even with optimal alpha, the perplexity of trigram is much larger than bigram. Thereby, bigram performs best with the given data.

###  <font color='orange'>Improvement on Backoff and Interpolation </font>  

Backoff and Interpolation both use kind of N-Gram 'Hierarchy'. In back-off, we only 'back-off' to lower-order n-Gram if we have zero evidence for a higher-order n-Gram. While in Interpolation, we mixed the probability estimations from all the n-Gram estimators.

The given 'StupidBackoff' is lack of normalization while the 'StupidBackoffNormalized' fixed this problem and gives perplexity improved to '239.9377346450501'. And 'InterpolatedLM' shows better performance with perplexity '188.57971535126373'. Based on the given 'InterpolatedLM', which linearly interpolated a bigram with a unigram.  'InterpolatedLM2' that combines trigram, bigram and unigram, the perplexity, given optimal lambdas is '164.220977639824'

###  <font color='orange'>work done so far for other Smoothing Methods</font>  
The perplexity turns out to be inf when the Katz Smoothing algorithm is applied to the n-Grams.

    This might due to following issues when applying Good Turing:

    The adjusted count r* might equal to zero if the number of n-grams seen r+1 times {n_(r+1)} equals to zero, which leads to the holes in the counts of counts. Also, n_r are quite noisy for high r. 

    Thereby, it should be better think of 
$$
r^*=(r+1)\frac{E[n_{r+1}]}{E[n_r]}
$$

    rather than
$$
r^*=(r+1)\frac{n_{r+1}}{n_r}
$$

To estimate the expectation, ML estimate can be used.


###  <font color='orange'>Find optimal parameters for language model</font>  

I used "nested for loop" to find an optimal parameters for the chosen language model. 
Firstly, the parameters are set with range(0,1,0.1), a rough range of target alpha can be achieved from this step. The range can thereby be reduced around the parameter sets with smallest perplexity, and the steps can be minished. For complex models, small changes in parameters can cause big changes in the result.

For a single n-gram, the optimization is simple that we just need one loop.

Considering Laplace Smoothing, it turns out that it cannot make much enhancement for this task; therefore, I just used 'LaplaceLM' for unigram.

Speaking of Interpolation Smoothing. When a bigram is linearly interpolated with a unigram, the discounting rate is found to be around '0.7167', given perplexity of '188.57971535126373'.
In order to estimate the trigram probability by mixing together the unigram, bigram and trigram probabilities, each weighted by a $\lambda $(the sum of this three lambda is 1), I wrote a simple 'InterpolatedLM2' class. The optimal $\lambda$ for trigram, bigram and unigram are '0.22'，‘0.52’ and '0.28' respectively. Given perplexity of '164.220977639824'.
Based on this, I tried to use both 'InterpolatedLM' and 'InterpolatedLM2' to see how it performs. The optimal $\lambda$ for quadgram, trigram, bigram and unigram are '0.1','0.1','0.1' and '0.7' respectively. Given perplexity of '156.12951366236243'. However, during the tuning it reflected that the perplexity was infinite with some parameter sets.
Therefore, I chose to use the given 'InterpolatedLM' for my final language model. The function was called for five times to mixed the estimator of 1-Gram to 6-Gram. And the final perplexity is '151.263077651845'.